In [11]:
import pandas as pd
import numpy as np
import os

# ===============================
# 1. Correct File Path (YOUR PATH)
# ===============================
input_file = r"C:\Users\tkorz\OneDrive\Documents\revenue-analysis-case-study\data\raw\case_study_original.xlsx"

output_file = r"C:\Users\tkorz\OneDrive\Documents\revenue-analysis-case-study\data\processed\case_study_cleaned.xlsx"

# Tabs to modify
tabs_to_modify = [
    "Recurring Revenue by Customer",
    "Recurring Revenue Waterfall"
]

# Validate file exists
if not os.path.exists(input_file):
    raise FileNotFoundError(f"File not found at: {input_file}")

# Load workbook
xls = pd.ExcelFile(input_file)
sheet_names = xls.sheet_names  # preserve original order

cleaned_sheets = {}

# ===============================
# 2. Process Sheets
# ===============================
for sheet in sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet)

    if sheet in tabs_to_modify:

        print(f"\nProcessing sheet: {sheet}")

        df_original = df.copy()
        df.columns = df.columns.str.strip().str.lower()

        # -------------------------------
        # A) Replace Missing Values
        # -------------------------------
        object_cols = df.select_dtypes(include=["object"]).columns
        df[object_cols] = df[object_cols].fillna("UNKNOWN")

        numeric_cols = df.select_dtypes(include=[np.number]).columns
        df[numeric_cols] = df[numeric_cols].fillna(0)

        # -------------------------------
        # B) Surrogate Key
        # -------------------------------
        df["surrogate_key"] = df.index.astype(str)

        key_columns = ["company", "customer", "date"]
        if all(col in df.columns for col in key_columns):

            composite_key = (
                df["company"].astype(str) + "_" +
                df["customer"].astype(str) + "_" +
                df["date"].astype(str)
            )

            df["surrogate_key"] = np.where(
                composite_key.str.contains("UNKNOWN"),
                df.index.astype(str),
                composite_key
            )

        # -------------------------------
        # C) Revenue Impact Analysis
        # -------------------------------
        if "value eur" in df.columns:
            df.rename(columns={"value eur": "revenue"}, inplace=True)

        if "revenue" in df.columns:

            total_revenue = df["revenue"].sum()

            if "customer" in df_original.columns:
                missing_mask = df_original["customer"].isna()
                unknown_revenue = df.loc[missing_mask, "revenue"].sum()

                impact_pct = (
                    (unknown_revenue / total_revenue) * 100
                    if total_revenue != 0 else 0
                )

                print("Total Revenue:", round(total_revenue, 2))
                print("Revenue from UNKNOWN customers:", round(unknown_revenue, 2))
                print("Revenue impact %:", round(impact_pct, 4))

        cleaned_sheets[sheet] = df

    else:
        # Leave other sheets unchanged
        cleaned_sheets[sheet] = df


# ===============================
# 3. Save File (Preserve Order)
# ===============================
os.makedirs(os.path.dirname(output_file), exist_ok=True)

with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    for sheet in sheet_names:
        cleaned_sheets[sheet].to_excel(writer, sheet_name=sheet, index=False)

print("\n✔ Cleaning complete.")
print("✔ File saved to:")
print(output_file)


Processing sheet: Recurring Revenue by Customer


C:\Users\tkorz\AppData\Local\Temp\ipykernel_37288\926588800.py:44: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  object_cols = df.select_dtypes(include=["object"]).columns



Processing sheet: Recurring Revenue Waterfall


C:\Users\tkorz\AppData\Local\Temp\ipykernel_37288\926588800.py:44: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  object_cols = df.select_dtypes(include=["object"]).columns



✔ Cleaning complete.
✔ File saved to:
C:\Users\tkorz\OneDrive\Documents\revenue-analysis-case-study\data\processed\case_study_cleaned.xlsx


In [16]:
import pandas as pd
import numpy as np

file_path = r"C:\Users\tkorz\OneDrive\Documents\revenue-analysis-case-study\data/raw/case_study_original.xlsx"

tabs = [
    "Recurring Revenue by Customer",
    "Recurring Revenue Waterfall"
]

results = []

for sheet in tabs:
    df = pd.read_excel(file_path, sheet_name=sheet)
    df.columns = df.columns.str.strip().str.lower()

    # Convert UNKNOWN back to NaN for analysis
    df.replace("UNKNOWN", np.nan, inplace=True)

    # Use 'value eur' instead of 'revenue'
    total_revenue = df["value eur"].sum()

    print(f"\nAnalyzing sheet: {sheet}")

    # Identify all columns with missing values (excluding value eur)
    missing_columns = df.columns[df.isna().any()].tolist()
    missing_columns = [col for col in missing_columns if col != "value eur"]

    print("Columns with missing values:", missing_columns)

    for col in missing_columns:

        missing_mask = df[col].isna()
        missing_revenue = df.loc[missing_mask, "value eur"].sum()

        impact_pct = (
            missing_revenue / total_revenue * 100
            if total_revenue != 0 else 0
        )

        if "category" in df.columns:

            breakdown = (
                df.loc[missing_mask]
                  .groupby("category")["value eur"]
                  .sum()
                  .reset_index()
            )

            breakdown["Sheet"] = sheet
            breakdown["Missing Column"] = col
            breakdown["Total Revenue"] = total_revenue
            breakdown["Revenue Impact %"] = (
                breakdown["value eur"] / total_revenue * 100
            )

            results.append(breakdown)

        else:
            results.append(pd.DataFrame({
                "Sheet": [sheet],
                "Missing Column": [col],
                "category": [None],
                "value eur": [missing_revenue],
                "Total Revenue": [total_revenue],
                "Revenue Impact %": [impact_pct]
            }))

final_impact_df = pd.concat(results, ignore_index=True)
final_impact_df



Analyzing sheet: Recurring Revenue by Customer
Columns with missing values: []

Analyzing sheet: Recurring Revenue Waterfall
Columns with missing values: ['customer']


,category,value eur,Sheet,Missing Column,Total Revenue,Revenue Impact %
0,Delta with reported RR,-6108.576114,Recurring Revenue Waterfall,customer,2.134998e+07,-0.028612
